In [ ]:
!pip install tensorflow

In [ ]:
import numpy
import tensorflow as tf
from numpy import array

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding

from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [ ]:
text = '아 집가고 싶다.'

In [ ]:
result = text_to_word_sequence(text)

print('원래 문구: ', text)
print('토큰화', result)

원래 문구:  아 집가고 싶다.
토큰화 ['아', '집가고', '싶다']


In [ ]:
docs = ['먼저 텍스트와 각 단어를 나누어 토큰화 합니다',
        '텍스트의 단어로 토큰화 해야 딥러닝에서 인식됩니다.',
        '토큰화 한 결과는 딥러닝에서 사용할 수 있습니다.',
        ]

In [ ]:
token = Tokenizer()
token.fit_on_texts(docs)

In [ ]:
print('단어 카운트', token.word_counts)

단어 카운트 OrderedDict([('먼저', 1), ('텍스트와', 1), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 3), ('합니다', 1), ('텍스트의', 1), ('단어로', 1), ('해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])


In [ ]:
print('문장 카운트: ', token.document_count)
print('각 단어가 몇개의 문장에 포함되는가: ', token.word_docs)
print('각 단어에 매겨진 인덱스 값: ', token.word_index)

문장 카운트:  3
각 단어가 몇개의 문장에 포함되는가:  defaultdict(<class 'int'>, {'합니다': 1, '토큰화': 3, '단어를': 1, '각': 1, '텍스트와': 1, '먼저': 1, '나누어': 1, '딥러닝에서': 2, '단어로': 1, '텍스트의': 1, '해야': 1, '인식됩니다': 1, '수': 1, '한': 1, '있습니다': 1, '결과는': 1, '사용할': 1})
각 단어에 매겨진 인덱스 값:  {'토큰화': 1, '딥러닝에서': 2, '먼저': 3, '텍스트와': 4, '각': 5, '단어를': 6, '나누어': 7, '합니다': 8, '텍스트의': 9, '단어로': 10, '해야': 11, '인식됩니다': 12, '한': 13, '결과는': 14, '사용할': 15, '수': 16, '있습니다': 17}


In [ ]:
docs = ['너무 재밌어요', '최고에요', '참 잘 만든 영화네요', '추천하고 싶은 영화 입니다.', '한번 더 보고싶네요',
        '글쎄요', '별로에요', '생각보다 지루하네요', '연기가 어색해요', '재미없어요']

In [ ]:
classes = array([1,1,1,1,1,0,0,0,0,0])

In [ ]:
token = Tokenizer()

In [ ]:
token.fit_on_texts(docs)

print(token.word_index)

{'너무': 1, '재밌어요': 2, '최고에요': 3, '참': 4, '잘': 5, '만든': 6, '영화네요': 7, '추천하고': 8, '싶은': 9, '영화': 10, '입니다': 11, '한번': 12, '더': 13, '보고싶네요': 14, '글쎄요': 15, '별로에요': 16, '생각보다': 17, '지루하네요': 18, '연기가': 19, '어색해요': 20, '재미없어요': 21}


어휘 사전
- 훈련 세트에서 고유한 단어를 뽑아 만든 목록
- 단어에 매핑한 정수는 단어의 의미와 크기와 무관
- 특수한 목적으로 예약된 정수
 - 0: 패딩
 - 1: 문장의 시작
 - 2: 어휘사전에 없는 토큰
 -- 테스트 세트 안에 어휘 사전에 없는 단어가 있다면 2로 반환

In [ ]:
x = token.texts_to_sequences(docs)

print('문장 숫자화: ', x)

문장 숫자화:  [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14], [15], [16], [17, 18], [19, 20], [21]]


In [ ]:
padded_x = pad_sequences(x, 10)

print('패딩 결과:\n', padded_x)

패딩 결과:
 [[ 0  0  0  0  0  0  0  0  1  2]
 [ 0  0  0  0  0  0  0  0  0  3]
 [ 0  0  0  0  0  0  4  5  6  7]
 [ 0  0  0  0  0  0  8  9 10 11]
 [ 0  0  0  0  0  0  0 12 13 14]
 [ 0  0  0  0  0  0  0  0  0 15]
 [ 0  0  0  0  0  0  0  0  0 16]
 [ 0  0  0  0  0  0  0  0 17 18]
 [ 0  0  0  0  0  0  0  0 19 20]
 [ 0  0  0  0  0  0  0  0  0 21]]


In [ ]:
word_size = len(token.word_index) + 1

**순환신경망 만들기** \
SimpleRNN클래스
- 순환 층을 구현한 클래스
- tensorflow.keras.layers 모듈에 포함
- units: 순환 층의 뉴런 수
- activations = 'tanh'

단어 임베딩의 구현
- Embedding 클래스
-- input_dim : 어휘 사전의 크기
-- output_dim : 임베딩 된 벡터의 크기
-- input_length = None 입력 시퀀스의 길이

In [ ]:
model = Sequential()

model.add(Embedding(word_size, 8, input_length = 10))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


model.fit(padded_x, classes, epochs = 20)


print('accuracy: %.4f' % (model.evaluate(padded_x, classes)[1]))

Epoch 1/20
1/1 [==============================] - 1s 610ms/step - loss: 0.6906 - accuracy: 0.7000
Epoch 2/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6891 - accuracy: 0.7000
Epoch 3/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6875 - accuracy: 0.7000
Epoch 4/20
1/1 [==============================] - 0s 13ms/step - loss: 0.6859 - accuracy: 0.7000
Epoch 5/20
1/1 [==============================] - 0s 13ms/step - loss: 0.6844 - accuracy: 0.8000
Epoch 6/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6828 - accuracy: 0.8000
Epoch 7/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6812 - accuracy: 0.8000
Epoch 8/20
1/1 [==============================] - 0s 17ms/step - loss: 0.6797 - accuracy: 0.8000
Epoch 9/20
1/1 [==============================] - 0s 14ms/step - loss: 0.6781 - accuracy: 0.8000
Epoch 10/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6765 - accuracy: 0.8000
Epoch 11/20
1/1 [===========